# 提取Humphrey视野数据

使用新的方式重构

# 设定文件路径参数

In [1]:
if __name__=="__main__":
    input_path='../../testdata/Humphrey'
    output_path="../../testdata/Humphrey"
    fname="huangzeyuan13.pdf"
    info_location_path='../../medical_device_data/'
    info_basic_fname="humphrey_basic_location.csv"
    info_30_2_fname="humphrey_30-2_location.csv"
    info_24_2_fname="humphrey_24-2_location.csv"
    info_fname_dict={"basic": info_basic_fname, "30-2":info_30_2_fname, "24-2":info_24_2_fname}
#     fname="右眼-sfa_zh.pdf" # for test

## 导入依赖包

在使用notebook.azure.com在线运行时, 由于默认没有安装pdfminer.six这个包, 所以在首次运行时需要安装, 已经将安装代码加入到下面导入依赖包的代码内, 因此首次运行时速度会较慢. 

同时, 在使用notebook.azure.com在线运行时, 服务器端不会保存曾经安装过的包, 因此在1小时没有操作之后, 服务器会关闭, 再次打开时就已经丢失了之前安装的包, 相当于首次运行. 

In [2]:
import sys
import os
import os.path
import io
import re
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

try:
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.pdfpage import PDFPage
    from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
    from pdfminer.layout import LAParams
except:
    !conda install pdfminer.six --yes
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.pdfpage import PDFPage
    from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
    from pdfminer.layout import LAParams


## 导入 PDF_parser_by_location 
PDF_parser_by_location 中将所有PDF转换成带有html, 其中每个字符均有定位, 通过选取一个方框来对一个数据或者单词进行选择. 各个数据的定位数据放置在相应的csv文件中, 由info_lation_path和info_fname保存

In [3]:
from PDF_parser_by_location import read_data_from_location, pdf_parser

# 读取

In [4]:
def read_one_Humphrey_data(input_path, fname, info_location_path, info_fname):
    page_number=0
    df=read_data_from_location(input_path, fname, info_location_path, info_fname, page_number)
    df=df.set_index("item_name")
    return df.T

In [5]:
def get_humphrey_test_method(input_path, fname, info_location_path, info_fname_dict):
    df=read_one_Humphrey_data(input_path, fname, info_location_path, info_fname_dict["basic"])
    return df.Test.values[0]

In [6]:
def get_full_humphrey_data(input_path, fname, info_location_path, info_fname_dict):
    humphrey_test_method=get_humphrey_test_method(input_path, fname, info_location_path, info_fname_dict)
    for key in info_fname_dict.keys():
        if key in humphrey_test_method: 
            info_fname=info_fname_dict[key]
    df=read_one_Humphrey_data(input_path, fname, info_location_path, info_fname)
    return df

In [7]:
# if __name__=="__main__":
#     df=get_full_humphrey_data(input_path, fname, info_location_path, info_fname_dict)


## 处理目录

In [32]:
if __name__=="__main__":

    pdffiles = [name for name in os.listdir(input_path)
            if name.endswith('.pdf')]
    df=DataFrame()
    for fname in pdffiles:
        newdf=get_full_humphrey_data(input_path, fname, info_location_path, info_fname_dict)
        df=df.append(newdf, sort=False)
        

In [37]:
# df.iloc[:,24:34]
df

,patient,date of birth,gender,patient ID,eye,date,Test,False POS Errors,False NEG Errors,VFI%,...,O7_1,O7_2,O7_3,O7_4,O7_5,O7_6,O8_1,O8_2,O8_3,O8_4
string_value,"CHEN HAO,","Oct 19, 1975",Other,12082202,OD,"Aug 22, 2012",Central 30-2 Threshold Test,6%,0%,99%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
string_value,"CHEN HAO,","Oct 19, 1975",Other,12082202,OD,"Sep 13, 2016",Central 30-2 Threshold Test,11%,4%,92%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
string_value,"CHEN HAO,","Oct 19, 1975",Other,12082202,OD,"Dec 12, 2016",Central 30-2 Threshold Test,12%,0%,90%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
string_value,"dong jia,",1995/2/28,其他,1995.0228.5268.6,OD,2016/6/14,中央 24-2 阙值测定,0%,0%,99%,...,31,33,34,33,33,29,31,33,32,30
string_value,"rong zheng-zheng,","Oct 10, 1978",Other,1978.1010.97FF,OD,"Jun 26, 2017",Central 24-2 Threshold Test,15% X,5%,76%,...,31,32,33,33,32,27,32,32,30,30
string_value,"rong zheng-zheng,",1978/10/10,其他,1978.1010.97FF.,OS,2017/6/26,中央 24-2 阙值测定,11%,1%,85%,...,30,32,33,33,33,33,30,30,32,30
string_value,"liu zhi-bo,",1983/7/1,其他,1983.0701.162E.,OD,2017/7/24,中央 24-2 阙值测定,22% X,9%,87%,...,28,35,34,35,33,31,30,32,33,33
string_value,"liu zhi-bo,",1983/7/1,其他,1983.0701.162E.,OS,2017/7/24,中央 24-2 阙值测定,11%,0%,100%,...,33,32,33,33,32,33,30,32,32,31
string_value,"liu qing,",1969/6/12,女性,1969.0612.F9CD,OD,2017/4/7,中央 24-2 阙值测定,1%,0%,84%,...,32,32,33,31,32,32,31,32,31,32
string_value,"liu qing,","Jun 12, 1969",Female,1969.0612.F9CD,OS,"Apr 07, 2017",Central 24-2 Threshold Test,0%,4%,77%,...,27,30,29,28,28,27,30,29,26,26
